In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os
from PIL import Image
print(os.listdir("./common/data"))

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# Keras Libraries
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

['chest_xray', 'resampled', 'TEST']


Using TensorFlow backend.


# Resample Data Into New Dataset

# Resample Train Data

In [2]:
from common.src import resample as res

In [3]:
from imblearn.under_sampling import EditedNearestNeighbours

path = r'C:\Users\mmitk\dev\2020\pneumonia\common\data\chest_xray\train'
ENN = EditedNearestNeighbours()

In [4]:
res.resample_directory(ENN, path, 'ENN', val = False)

# Load Data

In [5]:
train_folder= './common/data/resampled/ENN/train'
val_folder = './common/data/chest_xray/val/'
test_folder = './common/data/chest_xray/test/'

In [6]:
# train 
os.listdir(train_folder)
train_n = train_folder+'NORMAL/'
train_p = train_folder+'PNEUMONIA/'

In [7]:
# test
os.listdir(test_folder)
train_n = test_folder+'NORMAL/'
train_p = test_folder+'PNEUMONIA/'

In [8]:
from common.src import util

train_datagen = util.create_train_datagen()
test_datagen = util.create_test_datagen()

training_set = util.create_generator_set(train_datagen, './common/data/resampled/ENN/train')
validation_generator = util.create_generator_set(test_datagen, './common/data/chest_xray/val/')
test_set = util.create_generator_set(test_datagen, './common/data/chest_xray/test/') 

Found 4963 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


# Create Model

In [9]:
from common.src import models

cnn = models.CNNModel()
cnn.create_model()
cnn.summary()

# Fit Model

In [ ]:
cnn_model = cnn.fit_generator(training_set, validation_generator,'./common/data/models/ENNe20.hdf5', epochs=20)

Epoch 1/20
162/163 [============================>.] - ETA: 0s - loss: 0.3483 - tp: 4348.0000 - fp: 791.0000 - tn: 4364.0000 - fn: 807.0000 - accuracy: 0.8450 - precision: 0.8461 - recall: 0.8435 - auc: 0.9281

In [ ]:
cnn.plot_history()

In [ ]:
cnn.get_classification_report(test_set)

In [ ]:
cnn.display_confusion_matrix(test_set)